### Ingestion del archivo "person.json"

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
movies_genres_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("genreId", IntegerType(), True)
])

In [0]:
movies_genres_df = spark.read \
                    .schema(movies_genres_schema) \
                    .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

### Paso 2. Renombrar las columnas y añadir nuevas columnas

1. "movieId" renombrar a "movie_id"
1. "genreId" renombrar a "genre_id"
1. Agregar las columnas "ingestion_date" y "enviroment"

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
movies_genres_final_df = movies_genres_df \
                        .withColumnRenamed("movieId", "movie_id") \
                        .withColumnRenamed("genreId", "genre_id")

In [0]:
movies_genres_final_df = add_ingestion_date(movies_genres_final_df) \
                        .withColumn("enviroment", lit(v_enviroment)) \
                        .withColumn("file_date", lit(v_file_date))

### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movie_genre_final_df, 'movie_silver', 'movie_genres', 'file_date')

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date'
merge_delta_lake(movies_genres_final_df, "movie_silver", "movies_genres", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movies_genres
GROUP BY file_date;

In [0]:
display(spark.read.format('delta').load(f'{silver_folder_path}/movies_genres'))

In [0]:
dbutils.notebook.exit("Success")